In [11]:
_RIOT_API_KEY='RGAPI-f133c0d1-25f4-4ab2-ad9e-7d96aa3b73f5'
from riotwatcher import LolWatcher
import helper_functions as help
import role_classification as rc
import analysis_functions as af
import constants
import pandas as pd
import numpy as np
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
import concurrent.futures

from collections import Counter
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

from itertools import compress

lol_watcher = LolWatcher(_RIOT_API_KEY)

In [12]:
#match = lol_watcher.match.by_id('na1', 'NA1_3891774034')
#match = lol_watcher.match.by_id('na1', 'NA1_4720874733')
engine = help.create_mysql_engine()

# Plan for New Role Classification
1. Create new DB table 'lol_match_roles' with: matchId, participantId, Role, validRoles
2. Re-Create function for Jungle Flag
2. Re-Create function for Support Flag
3. Re-Create function for Lane By Posx/Posy
4. Create Mother Function 'determine_roles' To Validate positions for a match. validRoles = True if 1 player in each role, False otherwise. Upload to new 
5. Probably create a 'lol_analysis_view' which pulls needed info from 'lol_match_details', 'lol_match_timeline', and 'lol_match_roles' which an analysis script can pull from.


In [3]:
print(",".join([str(i) for i in [1]]))

1


In [78]:
df = af.prep_comparison_df('souporsecret', engine, minutes_to_analyse = [14], teammates_names = [])
pd.options.display.max_columns=100
df.head(3)

,matchId,championName_x,summonerName_x,role,winFlag_x,teamName_x,teamMate_x,queueName,gameCreationDate,minute,participantId_x,totalGold_x,xp_x,minionsKilled_x,jungleMinionsKilled_x,totalCS_x,totalDamageDoneToChampions_x,championName_y,summonerName_y,winFlag_y,teamName_y,teamMate_y,participantId_y,totalGold_y,xp_y,minionsKilled_y,jungleMinionsKilled_y,totalCS_y,totalDamageDoneToChampions_y,cs_diff,xp_diff,gold_diff,dmg_diff
0,NA1_4630778042,Zyra,souporsecret,SUPPORT,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,5,3552,3337,11,0,11,4275,Milio,1 tô bún bò hue,0,Red,0,10,2986,3841,1,0,1,994,10,-504,566,3281
1,NA1_4630778042,Draven,aerialblast,BOTTOM,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,4,4183,4391,100,0,100,841,Ashe,darkness197,0,Red,0,9,4611,4523,104,0,104,4470,-4,-132,-428,-3629
2,NA1_4630778042,KSante,ncm shrek,MIDDLE,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,3,4210,7146,100,0,100,6404,Gwen,dragonsins2811,0,Red,0,6,4150,6283,89,4,93,6079,7,863,60,325


In [79]:
cols=['role','championName_x','championName_y','winFlag_x','minute','cs_diff','xp_diff','gold_diff','dmg_diff']
mid_df_10 = df[(df.summonerName_x=='souporsecret')][cols]
#  & (df.gameCreationDate > pd.Timestamp('2022-01-01 15:48:49'))

In [81]:

agg_types={'championName_y':'count','winFlag_x':'mean', 'cs_diff':'mean','xp_diff':'mean', 'gold_diff':'mean', 'dmg_diff':'mean' }
rename_dict = {'championName_y':'Games', 'winFlag_x':'Winrate', 'cs_diff':'CS_Diff@14', 'xp_diff':'XP_Diff@14', 'gold_diff':'Gold_Diff@14', 'dmg_diff':'DMG_Diff@14'}
x=mid_df_10.groupby('championName_x', as_index=False,).agg(agg_types).rename(columns=rename_dict).sort_values(by='Games', ascending=False)
format_dict={'Game': "{:.0f}",
                'Winrate': "{:.0%}",
                'CS_Diff@14': "{:.0f}",
                'XP_Diff@14': "{:.0f}",
                'Gold_Diff@14': "{:.0f}",
                'DMG_Diff@14': "{:.0f}"}
x.style.format(format_dict)

,championName_x,Games,Winrate,CS_Diff@14,XP_Diff@14,Gold_Diff@14,DMG_Diff@14
35,Leona,45,51%,8,43,107,-611
78,Xerath,40,60%,1,93,187,1650
85,Zyra,38,58%,-2,237,728,2928
42,MissFortune,34,62%,8,35,94,1461
30,Kennen,34,53%,2,82,96,2074
16,Ezreal,33,52%,6,208,457,1858
57,Seraphine,31,68%,-9,-17,182,857
65,Syndra,29,66%,1,-194,192,757
34,LeeSin,27,48%,-5,125,128,1108
12,Corki,23,57%,5,-48,340,1365


In [19]:
def stats_at_min(summoner_name, role, db_engine, minutes_to_analyse =[14], teammates_names=[],
                 beg_timestamp= pd.Timestamp('2019-01-01 15:48:49'), end_timestamp= pd.Timestamp('2025-12-31 15:48:49'), ):
    df = af.prep_comparison_df(summoner_name, db_engine, minutes_to_analyse = minutes_to_analyse, teammates_names = teammates_names)
    cols=['role','championName_x','championName_y','winFlag_x','minute','cs_diff','xp_diff','gold_diff','dmg_diff']
    mid_df_10 = df[(df.summonerName_x==summoner_name) & (df.role==role) &
                   (df.gameCreationDate > beg_timestamp) &
                   (df.gameCreationDate < end_timestamp)][cols]
    
    agg_types={'championName_y':'count','winFlag_x':'mean', 'cs_diff':'mean','xp_diff':'mean', 'gold_diff':'mean', 'dmg_diff':'mean' }
    rename_dict = {'championName_y':'Games', 'winFlag_x':'Winrate', 'cs_diff':'CS_Diff@14', 'xp_diff':'XP_Diff@14', 'gold_diff':'Gold_Diff@14', 'dmg_diff':'DMG_Diff@14'}
    x=mid_df_10.groupby('championName_x', as_index=False,).agg(agg_types).rename(columns=rename_dict).sort_values(by='Games', ascending=False)
    format_dict={'Game': "{:.0f}",
                    'Winrate': "{:.0%}",
                    'CS_Diff@14': "{:.0f}",
                    'XP_Diff@14': "{:.0f}",
                    'Gold_Diff@14': "{:.0f}",
                    'DMG_Diff@14': "{:.0f}"}
    return x.style.format(format_dict)
stats_at_min('spicedcider', 'SUPPORT', engine, minutes_to_analyse =[14], teammates_names=[])
#stats_at_min('irishguy', 'JUNGLE', engine, minutes_to_analyse =[14], teammates_names=['souporsecret', 'radaemon1','spicedcider','aerialblast'])

,championName_x,Games,Winrate,CS_Diff@14,XP_Diff@14,Gold_Diff@14,DMG_Diff@14
0,Blitzcrank,25,68%,5,-20,-234,-948
1,Brand,21,71%,-2,-147,54,1863
6,Morgana,9,44%,4,-356,110,-357
12,Sion,8,25%,14,82,-247,896
9,Quinn,4,75%,-5,-546,193,197
11,Singed,4,50%,22,-315,-446,-2596
4,JarvanIV,2,0%,4,-536,-1070,-1034
10,Shen,2,100%,12,478,1274,1146
2,Braum,1,0%,14,-1532,-1062,-1717
3,Heimerdinger,1,100%,-1,82,1261,3528


In [20]:

stats_at_min('spicedcider', 'SUPPORT', engine, minutes_to_analyse =[25], teammates_names=[])


,championName_x,Games,Winrate,CS_Diff@14,XP_Diff@14,Gold_Diff@14,DMG_Diff@14
0,Blitzcrank,17,65%,5,176,-27,-593
1,Brand,13,77%,-3,-48,9,5367
6,Morgana,8,50%,12,-132,401,1137
12,Sion,6,33%,26,-204,-372,-935
11,Singed,3,67%,38,396,144,-1192
4,JarvanIV,2,0%,10,-2090,-1664,-3097
9,Quinn,2,50%,2,36,130,-3230
2,Braum,1,0%,20,-535,-870,3550
3,Heimerdinger,1,100%,8,861,1170,8896
5,LeeSin,1,100%,27,-555,810,7373


In [7]:
match_list_to_classify = ['asfasds','asdadfs','asdfsdf']
print("'" + "','".join([str(i) for i in match_list_to_classify]) + "'")
    

'asfasds','asdadfs','asdfsdf'


In [9]:
import analysis_functions as af

In [96]:
teammate_dict = {'souporsecret':'MIDDLE', 'aerialblast': 'ALL ROLES', 'radaemon1':'ALL ROLES'}


In [102]:
df = af.prep_comparison_df('souporsecret', engine, minutes_to_analyse = [14]) 
teammate_dict = {'souporsecret':'MIDDLE', 'aerialblast': 'ALL ROLES', 'radaemon1':'ALL ROLES'}
main_summoner_name = 'souporsecret'

beg_timestamp= pd.Timestamp('2018-01-01 15:48:49')
end_timestamp= pd.Timestamp('2025-12-31 15:48:49')
cols=['role','championName_x','championName_y','winFlag_x','minute','cs_diff','xp_diff','gold_diff','dmg_diff']

# Let's say we're looking for 3 summoners. There will be 1 match_id  for each  correct summoner_name / role combo.
# So then we need to select only the match_ids with 3 instances.
match_id_mask = [ m_id for m_id, summ_name, role in zip(df.matchId, df.summonerName_x, df.role) 
                       if summ_name in teammate_dict.keys() and (role == teammate_dict[summ_name] or teammate_dict[summ_name] == 'ALL ROLES') ]

val_counts = Counter(match_id_mask)
valid_matchlist = [m_id for m_id in val_counts if val_counts[m_id]==len(teammate_dict.keys())]
df = df[df['matchId'].isin(valid_matchlist)]

mid_df_10 = df[(df.summonerName_x == main_summoner_name) &
               (df.gameCreationDate > beg_timestamp) &
               (df.gameCreationDate < end_timestamp)][cols]




agg_types={'championName_y':'count','winFlag_x':'mean', 'cs_diff':'mean','xp_diff':'mean', 'gold_diff':'mean', 'dmg_diff':'mean' }
rename_dict = {'championName_y':'Games', 'winFlag_x':'Winrate', 'cs_diff':'CS_Diff@14', 'xp_diff':'XP_Diff@14', 'gold_diff':'Gold_Diff@14', 'dmg_diff':'DMG_Diff@14'}
x=mid_df_10.groupby('championName_x', as_index=False,).agg(agg_types).rename(columns=rename_dict).sort_values(by='Games', ascending=False)
format_dict={'Game': "{:.0f}",
                'Winrate': "{:.0%}",
                'CS_Diff@14': "{:.0f}",
                'XP_Diff@14': "{:.0f}",
                'Gold_Diff@14': "{:.0f}",
                'DMG_Diff@14': "{:.0f}"}
x.style.format(format_dict)

,championName_x,Games,Winrate,CS_Diff@14,XP_Diff@14,Gold_Diff@14,DMG_Diff@14
9,Syndra,10,80%,-2,-239,177,1345
10,Xerath,5,80%,-18,-779,-601,877
0,Ahri,3,100%,-5,140,1140,2519
2,Cassiopeia,3,33%,-4,-438,-320,527
7,Orianna,3,0%,2,-596,-177,1694
12,Zoe,3,33%,1,-30,338,905
1,Annie,2,100%,-7,-613,-134,2041
3,Karma,2,100%,11,839,300,2990
5,Leblanc,2,50%,-28,-1621,-1922,-2040
4,Kennen,1,0%,-28,-776,-293,1646


In [91]:
d={'aerialblast': 'ALL ROLES', 'radaemon1':'ALL ROLES'}
# def validate_teammates_role(summ_name, role, target_role):
#     if role == 'ALL ROLES':
#         return True
#     else:
#         return role == target_role
x=[ m_id for m_id, summ_name, role in zip(df.matchId, df.summonerName_x, df.role) 
         if summ_name in d.keys() and (role == d[summ_name] or d[summ_name] == 'ALL ROLES') ]
val_counts = Counter(x)
y=[m_id for m_id in val_counts if val_counts[m_id]==len(d.keys())]
y

['NA1_4630778042',
 'NA1_4630806218',
 'NA1_4630834311',
 'NA1_4635401379',
 'NA1_4635454820',
 'NA1_4639882367',
 'NA1_4639914136',
 'NA1_4656076300',
 'NA1_4656129977',
 'NA1_4656178177',
 'NA1_4669861855',
 'NA1_4669900271',
 'NA1_4670791781',
 'NA1_4670840155',
 'NA1_4670880032',
 'NA1_4677017498',
 'NA1_4695928006',
 'NA1_4695971814',
 'NA1_4710118987',
 'NA1_4710158772',
 'NA1_4710191130',
 'NA1_4710238204',
 'NA1_4716002274',
 'NA1_4716036182',
 'NA1_4716080477',
 'NA1_4720874733',
 'NA1_4071624131',
 'NA1_4598180873',
 'NA1_4286212921',
 'NA1_4441367376',
 'NA1_4598338742',
 'NA1_3830529253',
 'NA1_4045545894',
 'NA1_4356639594',
 'NA1_4427268815',
 'NA1_4041829156',
 'NA1_4516319607',
 'NA1_3840383384',
 'NA1_4485411045',
 'NA1_3817730265',
 'NA1_3953221723',
 'NA1_4333101415',
 'NA1_4356715902',
 'NA1_3804330594',
 'NA1_3844020687',
 'NA1_3948510297',
 'NA1_3860182640',
 'NA1_3834810362',
 'NA1_4278786427',
 'NA1_4286978602',
 'NA1_4627743186',
 'NA1_4403864185',
 'NA1_421379

In [97]:
len(d.keys())
df=df[df['matchId'].isin(y)]
df

,matchId,championName_x,summonerName_x,role,winFlag_x,teamName_x,teamMate_x,queueName,gameCreationDate,minute,...,totalGold_y,xp_y,minionsKilled_y,jungleMinionsKilled_y,totalCS_y,totalDamageDoneToChampions_y,cs_diff,xp_diff,gold_diff,dmg_diff
0,NA1_4630778042,Zyra,souporsecret,SUPPORT,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,...,2986,3841,1,0,1,994,10,-504,566,3281
1,NA1_4630778042,Draven,aerialblast,BOTTOM,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,...,4611,4523,104,0,104,4470,-4,-132,-428,-3629
2,NA1_4630778042,KSante,ncm shrek,MIDDLE,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,...,4150,6283,89,4,93,6079,7,863,60,325
3,NA1_4630778042,Skarner,irishguy,JUNGLE,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,...,3825,4766,2,72,74,529,9,-142,502,714
4,NA1_4630778042,Kayle,radaemon1,TOP,1,Blue,1,5v5 Ranked Flex games,2023-04-15 15:48:49,14,...,4298,7629,93,0,93,4208,16,-955,307,635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3790,NA1_3834718751,Leona,souporsecret,SUPPORT,1,Red,1,Clash games,2021-03-20 19:01:19,14,...,2954,3887,3,0,3,1154,14,-350,-15,151
3791,NA1_3834718751,Urgot,irishguy,MIDDLE,1,Red,1,Clash games,2021-03-20 19:01:19,14,...,3968,6565,96,0,96,2030,12,264,505,284
3792,NA1_3834718751,Hecarim,pepegamindset,JUNGLE,1,Red,1,Clash games,2021-03-20 19:01:19,14,...,4937,5706,10,96,106,1365,-12,-726,-567,634
3793,NA1_3834718751,Tristana,aerialblast,BOTTOM,1,Red,1,Clash games,2021-03-20 19:01:19,14,...,4131,3803,76,0,76,4298,6,541,568,-595


In [89]:
975 /5

195.0